In [2]:
'''
From Gerardo's 'preprocessing.py'
'''
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import logging

nltk.download('stopwords')
stemer = PorterStemmer()
tokenizer = TweetTokenizer()

def stem(tokens=''):
    return [stemer.stem(w) for w in tokens]
def tokenize(text=''):
    return tokenizer.tokenize(text)
def remove_stopwords(tokens=[]):
    stop_words = set(stopwords.words('english'))
    return [w for w in tokens if not w in stop_words]
def remove_capitalization(tokens=[]):
    return [w.lower() for w in tokens]
def remove_punctuation(text=''):
    return re.sub(r'[^\w\s]', '', text)

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
'''
pre-made sample inverted_index
'''
# path = "Documents/Classes/CS537_IR/project1/"
# with open('inverted_index_chunk1.pkl', 'rb') as handle:
#     b = pickle.load(handle)

'\npre-made sample inverted_index\n'

# Process query **q** using same techniques as index.

In [3]:
q = "what time is shark week"

In [4]:
def preprocess(text=''):
    # Remove punctuation from the text.
    text = remove_punctuation(text=text)
    # Tokenize
    tokens = tokenize(text=text)
    # Remove stop words
    tokens = remove_stopwords(tokens=tokens)
    # Remove capitalization
    tokens = remove_capitalization(tokens=tokens)
    # Stem terms
    tokens = stem(tokens=tokens)
    return tokens

In [5]:
tokens = preprocess(q)
tokens

['time', 'shark', 'week']

# create set of docs [**CR**] comprised of all docs in Document Collection [**DC**] that contains ALL of the terms in the query [**q**]. 

In [6]:
import pandas as pd
# my BS
prefix = "~/Documents/Classes/CS537_IR/project1/chunks/"
sample_path = prefix + 'chunk_1.pkl' 
doc_collection = pd.read_pickle(sample_path)

# from Gerardo
json_collection = doc_collection.to_dict('records')

In [7]:
dc = json_collection

## Turn DC into our index

In [8]:
from tqdm import tqdm

'''
from Gerardo's class inverted_index
'''
class inverted_index:
    
    def __init__(self, storage = ''):
        self.index = {}
#         self.storage = storage

    def __repr__(self):
        """
        String representation of the Database object
        """
        return str(self.index)

    def index_document(self, document=''):
        """
        Process a given document, save it to the DB and update the index.
        """
        tokens = preprocess(document['content'])

        appearances_dict = dict()
        # Dictionary with each term and the frequency it appears in the text.
        for term in tokens:
            term_frequency = appearances_dict[term].frequency if term in appearances_dict else 0
            appearances_dict[term] = Appearance(document['id'], term_frequency + 1)
        # Update the inverted index
        update_dict = {key: [appearance]
        if key not in self.index
        else self.index[key] + [appearance]
                       for (key, appearance) in appearances_dict.items()}
        self.index.update(update_dict)
        # Add the document into the database
#         self.storage.add(document)

        return document

    def lookup_query(self, query):
        """
        Returns the dictionary of terms with their correspondent Appearances.
        This is a very naive search since it will just split the terms and show
        the documents where they appear.
        """   
        return {term: self.index[term] for term in query if term in self.index}

    def create_index(self, collection=[]):
        for i, doc in tqdm(enumerate(collection)):
            self.index_document(document=doc)

In [9]:
class Appearance:
    """
    Represents the appearance of a term in a given document, along with the
    frequency of appearances in the same one.
    """

    def __init__(self, docId, frequency):
        self.docId = docId
        self.frequency = frequency

    def __repr__(self):
        """
        String representation of the Appearance object
        """
        return str(self.__dict__)

In [10]:
'''
running this on a 100,000 doc chunk of the dataset
'''
# i_i = inverted_index()
# i_i.create_index(dc)

'\nrunning this on a 100,000 doc chunk of the dataset\n'

## reminder: create set of docs [**CR**] comprised of all docs in Document Collection [**DC**] that contains ALL of the terms in the query [**q**]. 

In [ ]:
CR = i_i.lookup_query(tokens)

In [1]:
def return_matches(CR={}, len_DC=int):
    doc_first = {}
    for i in range(1, len_DC+1):
        doc_first[i] = []    

    for token in CR:
        for val in CR[token]:
            match = ('match: ' + token, 'freq: ' + str(val.frequency))
            doc_first[val.docId].append(match)

    return {k: doc_first[k] for k in doc_first if len(doc_first[k]) > 0}

In [ ]:
CR = return_matches(srtd, len(dc))
len(CR)

In [12]:
def match_scaling(CR = {}, match_num = int):
    return {k: CR[k] for k in CR if len(CR[k]) == match_num}

In [ ]:
len(match_scaling(CR, 3))

# If **CR > 50**, consider the docs in the Document Collection with **n-1** terms in the query ("all the combinations").

In [ ]:
len(match_scaling(CR, 2))